# Hyper parameter tunning
- updated 0921
- data  
7th_FE_ver4  
catboost 튜닝
- optuna, bayesian

## y_predict 값들 피클로 저장해놓기   
stacking & scaler 구현


## ready

In [ ]:
cd /content/drive/My Drive/빅콘

/content/drive/My Drive/빅콘


In [ ]:
!apt-get update --qq
!apt-get install fonts-nanum* -qq

E: Command line option --qq is not understood in combination with the other options
Selecting previously unselected package fonts-nanum.
(Reading database ... 144676 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-6_all.deb ...
Unpacking fonts-nanum-eco (1.000-6) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20170925-1_all.deb ...
Unpacking fonts-nanum-extra (20170925-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-1_all.deb ...
Unpacking fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-extra (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Setting up fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-eco (1.000-6) ...
Processing triggers for fontconfig 

In [ ]:
# 한글 폰트
import matplotlib.font_manager as fm
fm._rebuild()

In [ ]:
# # CUDA
# !pip install mxnet-cu80
# import mxnet as mx

In [ ]:
# ready
import os
import sys
import joblib
import pickle
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd 
import numpy as np
import random
import math

from IPython.display import display
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.max_info_columns', 500)

import imp
import datetime
from tqdm import tqdm

In [ ]:
# vusualize
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
plt.rc('font', family='NanumGothic')
plt.rc('axes', unicode_minus=False)

In [ ]:
# sklearn
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import accuracy_score

## Load Data
6th_FE_ver4

data['X'] / data['y']

In [ ]:
# load data
data = joblib.load(os.path.join('data', '7th_train_FE.pkl'))
locals().update(data)

# X, y 설정
X = data['X'] ; y = data['y'] ; log_y = np.log(y)

In [ ]:
# na
X.isnull().sum().sum()

3733

In [ ]:
# 확인
#X.info()

In [ ]:
# fill na
X0 = X.fillna(0)

# One Hot
#X_dum = pd.get_dummies(X)
X.shape#, X_dum.shape

(35379, 402)

### MAPE + CV function

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
# mape
def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
# mape with exp
def MAPE_exp(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((np.exp(y_true) - np.exp(y_pred)) / np.exp(y_true))) * 100

# Stacking + SHAP
#### CV + Save Prediction + SHAP

In [ ]:
# stacking with cv=5
# MAPE_exp_cv 구현

def MAPE_exp_st(model, X, y, cv_splits, version, scaling=False):        
    # setting
    mape = {'val_mape' : [], 'test_mape' : [],'final_mape' : []}
    pred = {'val_idx' : [], 'val_pred' : [],
            'test_idx' : [], 'test_pred' : [],
            'final_pred' : []}      # final : test set mean 값
    
    # split train, test
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=77)
    pred['test_idx'].append(X_test.index)

    # for 문 돌리기
    cv=KFold(n_splits=cv_splits,random_state=77,shuffle=True)
    for t,v in cv.split(X_train):
        X_train_ = X_train.iloc[t]       # 훈련용
        X_val_ = X_train.iloc[v]         # 검증용 분리.
        y_train_ = y_train.iloc[t]
        y_val_ = y_train.iloc[v]

        pred['val_idx'].append(v)

        # scaling : MinMax or Standard
        if scaling : 
            if scaling == 'MinMax':
                scaler = MinMaxScaler()
            elif scaling == 'Standard':
                scaler = StandardScaler()
            # fitting
            X_train_ = scaler.fit_transform(X_train_)
            X_val_ = scaler.transform(X_val_)
            X_test = scaler.transform(X_test)
        
        else:
            X_test = X_test

        # modeling
        model.fit(X_train_,y_train_)
        y_pred = model.predict(X_val_)
        pred['val_pred'].append(np.exp(y_pred))

        test_pred = model.predict(X_test)
        pred['test_pred'].append(np.exp(test_pred))

        # mape
        mape['val_mape'].append(MAPE_exp(y_val_, y_pred))
        mape['test_mape'].append(np.exp(test_pred))

    # final values
    final_test = np.mean(pred['test_pred'], axis=0)
    final_mape = MAPE_exp(y_test, final_test)
    pred['final_pred'].append(final_test)
    mape['final_mape'].append(final_mape)
    
    # loss
    loss = - np.mean(mape)

    # save pickle
    with open('mape.pickle' + '_' + version, 'wb') as f:
        pickle.dump(mape, f, pickle.HIGHEST_PROTOCOL)
    with open('pred.pickle' + '_' + version, 'wb') as f:
        pickle.dump(pred, f, pickle.HIGHEST_PROTOCOL)

    return mape, pred, loss

## Tunning CatBoost

In [ ]:
!pip install catboost
from catboost import CatBoostRegressor

     |████████████████████████████████| 66.1MB 65kB/s 


In [ ]:
! pip install bayesian-optimization
from bayes_opt import BayesianOptimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp36-none-any.whl size=11685 sha256=040971069110bedce12588c15944f1110170e7ee1b8908c17f6a56d358cc66f1
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


##### CatBoost + Bayesian

In [ ]:
# Setting
X, y = X, log_y 
def catboost_eval(bagging_temperature ,    # 3h 30min
                  depth , 
                  learning_rate ,
                  min_data_in_leaf ,  
                  border_count):
    
    n_splits=5
    cv = KFold(n_splits=5, shuffle=True, random_state=77)
    mape = []
    predict = None

    params = {}
    params['iterations'] = 10000
    params['eval_metric'] = 'MAPE'
    params['random_seed'] = 77
    params['learning_rate'] = learning_rate
    params['min_data_in_leaf'] = int(round(min_data_in_leaf))
    params['depth'] = int(round(depth))
    params['border_count'] = int(round(border_count))
    params['bagging_temperature'] = int(round(bagging_temperature))

    for tr_ind, val_ind in cv.split(X , y):
        X_train = X.iloc[tr_ind]
        y_train = y.iloc[tr_ind]
        X_valid = X.iloc[val_ind]
        y_valid = y.iloc[val_ind]

        # scaling
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_valid = scaler.transform(X_valid)
        
        model = CatBoostRegressor(**params , 
                                 leaf_estimation_iterations = 10,
                                 use_best_model=True,
                                 od_type="Iter",
                                )
        model.fit(X_train, 
                  y_train,
                  eval_set=(X_valid, y_valid),
                  verbose = False
                  )
        
        y_pred = model.predict(X_valid)
        mape.append(MAPE_exp(y_pred,y_valid))
    return -np.mean(mape)

catBO = BayesianOptimization(catboost_eval,
                             {'bagging_temperature': (0, 1000),
                              'depth': (5, 8.99) ,
                              "learning_rate" : (0.001,0.1) , 
                              "min_data_in_leaf" : (1,6) , 
                              'border_count': (5, 255) ,
                             },
                             random_state=0)     # 77

catBO.maximize(init_points=10, n_iter=20)


|   iter    |  target   | baggin... | border... |   depth   | learni... | min_da... |
-------------------------------------------------------------------------------------


In [ ]:
# best parameter
catBO.max

{'params': {'bagging_temperature': 639.9210213275238,
  'border_count': 40.8383218522616,
  'depth': 8.76922897902784,
  'learning_rate': 0.0526629838532571,
  'min_data_in_leaf': 3.0733096999526177},
 'target': -28.983983117259555}

##### CatBoost + OPTUNA  
- bagging_temperature': (0, 1000),
- depth': (5, 8.99) ,
- learning_rate" : (0.001,0.1) , 
- min_data_in_leaf" : (1,6) , 
- border_count': (5, 255) ,

In [ ]:
! pip install optuna
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [ ]:
# 2번째 방법
def objective(trial, X, y):
    
    # 파라미터들의 search 범위
    params = {
        'depth': trial.suggest_int('depth', 5, 10), # (5, 8.99)
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1,10),  
        'learning_rate': trial.suggest_uniform('learning_rate', 0.001, 0.1),
        'border_count': trial.suggest_int('border_count', 1, 260),
        'bagging_temperature': trial.suggest_int('bagging_temperature', 1, 1000),  
        'objective': 'regression'
    }

    # catboost setting
    cb_model = CatBoostRegressor(**params)
    
    # cat boost : split + cv
    mape, pred = MAPE_exp_st(cb_model, X, log_y, 5, '0920', 'MinMax')
    return np.mean(mape['test_mape'],axis=0)#[0]

In [ ]:
cat_study = optuna.create_study()
cat_study.optimize(lambda x: objective(x, X, y), timeout=1000, n_jobs=-1)
print(cat_study.best_params, cat_study.best_value)

[W 2020-09-20 13:18:08,818] Trial 5 failed because of the following error: CatBoostError('catboost/private/libs/options/loss_description.cpp:17: regression loss is not supported',)
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/optuna/study.py", line 778, in _run_trial
    result = func(trial)
  File "<ipython-input-82-0dd7e094986c>", line 2, in <lambda>
    cat_study.optimize(lambda x: objective(x, X, y), timeout=1000, n_jobs=-1)
  File "<ipython-input-81-7c3d9f4743bd>", line 18, in objective
    mape, pred = MAPE_exp_st(cb_model, X, log_y, 5, '0920', 'MinMax')
  File "<ipython-input-69-3c38ebd7f5dc>", line 40, in MAPE_exp_st
    model.fit(X_train_,y_train_)
  File "/usr/local/lib/python3.6/dist-packages/catboost/core.py", line 4835, in fit
    self._check_is_regressor_loss(params['loss_function'])
  File "/usr/local/lib/python3.6/dist-packages/catboost/core.py", line 4969, in _check_is_regressor_loss
    is_regression = self._is_regression_objective

CatBoostError: ignored

 Trial 23 failed because of the following error: CatBoostError('catboost/private/libs/options/loss_description.cpp:17: regression loss is not supported',)
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/optuna/study.py", line 778, in _run_trial
    result = func(trial)
  File "<ipython-input-82-0dd7e094986c>", line 2, in <lambda>
    cat_study.optimize(lambda x: objective(x, X, y), timeout=1000, n_jobs=-1)
  File "<ipython-input-81-7c3d9f4743bd>", line 18, in objective
    mape, pred = MAPE_exp_st(cb_model, X, log_y, 5, '0920', 'MinMax')
  File "<ipython-input-69-3c38ebd7f5dc>", line 40, in MAPE_exp_st
    model.fit(X_train_,y_train_)
  File "/usr/local/lib/python3.6/dist-packages/catboost/core.py", line 4835, in fit
    self._check_is_regressor_loss(params['loss_function'])
  File "/usr/local/lib/python3.6/dist-packages/catboost/core.py", line 4969, in _check_is_regressor_loss
    is_regression = self._is_regression_objective(loss_function) or self._i